In [1]:
from scapy.all import * 
from scapy.all import ICMP, IP
import time
import statistics

Creamos una función ping que envía echo_request al host pasado por parámetro. La función devuelve un diccionario que contiene la siguiente información:

* dst: Host contactado
* response: True si se obtuvo respuesta; False en caso contrario
* rtt_ms: RTT en milisegundos (None si no se obtuvo respuesta)
* len: Longitud del paquete recibido (None si no se obtuvo respuesta)
* ttl: Time to live del paquete recibido (None si no se obtuvo respuesta)


In [2]:
def ping(host):

    data = dict()
    data["dst"] = host

    print(f"Pinging {host}...")
    packet = IP(dst=host)/ICMP()
    response = sr1(packet, timeout=10, verbose=False) # sr1: envío y espero una sola respuesta
    inicio = time.time()
    if response:
        fin = time.time()
        rtt_ms = (fin - inicio) * 1000
        print(f"Respuesta recibida de {host}")
        # response.show()
        # print(response.len)
        # print(f"Tiempo de respuesta: {total_ms} milisegundos")
        
        data["response"] = True
        data["rtt_ms"] = rtt_ms
        data["len"] = response.len
        data["ttl"] = response.ttl

    else:
        print(f"No se obtuvo respuesta de {host}")
        data["response"] = False
        data["rtt_ms"] = None
        data["len"] = None
        data["ttl"] = None
    

    return data

Probamos contactar a utdt.edu.ar

In [3]:
ping("utdt.edu.ar")

Pinging utdt.edu.ar...
Respuesta recibida de utdt.edu.ar


{'dst': 'utdt.edu.ar',
 'response': True,
 'rtt_ms': 0.0007152557373046875,
 'len': 28,
 'ttl': 56}

Ahora, creamos una función que recibe un set de destinos a contactar. Devuelve un diccionario que contiene:
* Cantidad de paquetes enviados
* Cantidad de paquetes recibidos
* Cantidad de paquetes perdidos
* Porcentaje de paquetes perdidos
* RTT promedio
* RTT mínimo
* RTT máximo
* Desvío estándar del RTT promedio A QUE SE REFIERE?


In [4]:
def ping_multiple_hosts(hosts):

    res = dict()

    sent_packets = len(hosts)
    received_packets = 0
    lost_packets = 0
    total_rtt = 0
    min_rtt = float('inf')
    max_rtt = float('-inf')
    
    for host in hosts:
        resultado = ping(host)

        if resultado["response"]: # Si se recibió respuesta
            received_packets += 1
            total_rtt += resultado["rtt_ms"]
            min_rtt = min(min_rtt, resultado["rtt_ms"])
            max_rtt = max(max_rtt, resultado["rtt_ms"])
        else:
            lost_packets += 1
    
    if received_packets > 0:
        average_rtt = total_rtt / received_packets

    res["hosts"] = hosts
    res["sent_packets"] = sent_packets
    res["received_packets"] = received_packets
    res["lost_packets"] = lost_packets
    res["loss_percentage"] = (lost_packets / sent_packets) 
    res["total_rtt"] = total_rtt
    res["min_time"] = min_rtt if min_rtt != float('inf') else None
    res["max_time"] = max_rtt if max_rtt != float('-inf') else None
    res["average_rtt"] = average_rtt if received_packets > 0 else None
    
    return res

In [10]:
destinos = {"utdt.edu.ar", "mit.edu", "ie.edu", "sydney.edu.au"}

ping_multiple_hosts(destinos)

Pinging sydney.edu.au...
Respuesta recibida de sydney.edu.au
Pinging utdt.edu.ar...
Respuesta recibida de utdt.edu.ar
Pinging ie.edu...
Respuesta recibida de ie.edu
Pinging mit.edu...
Respuesta recibida de mit.edu


{'hosts': {'ie.edu', 'mit.edu', 'sydney.edu.au', 'utdt.edu.ar'},
 'sent_packets': 4,
 'received_packets': 4,
 'lost_packets': 0,
 'loss_percentage': 0.0,
 'total_rtt': 0.00286102294921875,
 'min_time': 0.000476837158203125,
 'max_time': 0.0011920928955078125,
 'average_rtt': 0.0007152557373046875}